## Computational Workflow Schematic
![Computational Workflow Schematic](images/fig_3.png)

# Insulator classification and Band gap Regression models for Double Oxide Perovskites
We implement two models in this notebook.


## i) Classification model 
- To separate materials that have a bandgap greater than 0.5 eV (Insulators) from those that have a very narrow band gap ( < 0.5 eV).

## ii) Regression model 
- To predict the band gap of materials classified by (i) as insulators

We will first look at how we build these models and then look at applying them on unseen data to predict new materials

### Since we are working with large datasets, we use the python [pickle](https://docs.python.org/3/library/pickle.html) module to manage our dataset. So we load our data management packages and some utility functions we have written to prepare the data for the models.

In [ ]:
import pickle
import pandas as pd
import perovmldis.engine as en
from perovmldis.data_utilities.generate_data import generate_feature_labels, csv_to_json, prepare_data

###  Next, we load our training compounds and elemental data

In [ ]:
with open('data/training_compounds.pkl', 'rb') as f:
                training_compounds = pickle.load(f)
with open('data/element_data.pkl', 'rb') as f:
                ele_data = pickle.load(f)

In [ ]:
ele_data['Al'].keys()

In [ ]:
training_compounds[10].keys()

### Then, we populate the training data with all the features we have finalized

In [ ]:
dft_training_data = en.create_perovskite_stability_training_data(training_compounds, ele_data)

In [ ]:
training_compounds[10].keys()

In [ ]:
training_compounds[1200]

## Classification of oxide perovskites into narrow and wide band gap materials

### Use the training data to train the classification model. We also print out the performance metrics, confusion matrices (we use a 80/20 split for training and testing)  and plot the feature importances

In [ ]:
insulator_feature_list, test_features,test_labels,insulator_clf = en.run_insulator_classification(dft_training_data)

## Next, we analyze the performance of our classification model, my plotting the performance curves: 

i) The Receiver Operating Characteristic (ROC) curve

ii) Precision Recall (PR) curve

Ideally, you will spend some time fine tuning your model, adding more training data if necessary, until you are satisfied with the performance. Here we show our finished models.

In [ ]:
from perovmldis.ML_utilities.ML_plotting import plot_roc_curves
plot_roc_curves(test_features,test_labels, insulator_clf)

## Regression model to predict band gap  of oxide perovskites expected to have a wide band gap

### First, we extract the training data for band gap regression. We train the regression model only using those data which have a wide band gap ( > 0.5 eV)


In [ ]:
bandgap_training_data = en.create_bandgap_regression_training_data(training_compounds, ele_data)

## Plot the training data

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
plot_data = pd.DataFrame(bandgap_training_data)
fig, ax = plt.subplots()
x = plot_data['PBE_band_gap'].values
n, bins, patches = plt.hist(x, 50, facecolor='g', alpha=0.75)
plt.xlabel('PBE band gap (eV)',fontsize=12) 
plt.ylabel('Bin count',fontsize=12) 
plt.grid(linestyle='--', linewidth='0.5')

##  Use the training data to train the regression model. 

### We also print out the performance metrics and plot the feature importances, the parity curves for training and testing and the confidence intervals

We use a 80/20 split for training and testing

In [ ]:
regression_feature_list, best_estimator = en.run_bandgap_regression(bandgap_training_data)

## Predict new oxide perovskites with wide band gap

In [ ]:
all_candidates = csv_to_json('data/stable_formable_candidates.csv')
print(len(all_candidates))

### This is a very large dataset that contains ~500,000 compounds, too large to use in this notebook. So for the purpose of demonstration, we proceed with only 100,000 candidates.

In [ ]:
condensed_candidate_data = all_candidates[0:100000]
all_candidate_data = prepare_data(condensed_candidate_data,ele_data)

### To speed things up, we clear some memory by deleting some data we no longer need

In [ ]:
del dft_training_data 
del all_candidates
import gc
gc.collect()

In [ ]:
from perovmldis.ML_utilities.RFC_functions import classify_data
insulator_candidates = classify_data(all_candidate_data, insulator_feature_list, insulator_clf, pred_label='predicted_Insulator', data_type='prediction', model_type='insulator')

### Of the 1350216 candidates, 16950 are predicted to be insulators

In [ ]:
from perovmldis.ML_utilities.RFR_functions import predict_regression
wide_bandgap_data = predict_regression(insulator_candidates, regression_feature_list, best_estimator, pred_label='Predicted_band_gap')

In [ ]:
plot_predicted_data = pd.DataFrame(wide_bandgap_data)
fig, ax = plt.subplots()
x = plot_predicted_data['Predicted_band_gap'].values
n, bins, patches = plt.hist(x, 50, facecolor='g', alpha=0.75)
plt.xlabel('Predicted band gap (eV)',fontsize=12) 
plt.ylabel('Bin count',fontsize=12) 
plt.grid(linestyle='--', linewidth='0.5')

## More analysis: Partial Dependence Plots (PDPs)

In [ ]:
importances = list(best_estimator.feature_importances_)
feature_importances = [(feature, round(importance, 5)) for feature, importance in zip(regression_feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
ranked_features = []
for i in range(5):
    ranked_features.append(feature_importances[i][0])	
dataset = pd.DataFrame(bandgap_training_data)
ranked_labels = generate_feature_labels(ranked_features)
print(ranked_labels)

### We now plot the PDPs for the top three features identified by the regression model.

In [ ]:
from perovmldis.ML_utilities.ML_plotting import plot_pdp_plots
plot_pdp_plots(best_estimator,dataset,regression_feature_list,ranked_features[0:3],ranked_labels[0:3])